In [1]:
import numpy as np
import random

In [2]:
# Подписчики
class Subscriber:
    def __init__(self, id, num_channels):
        self.id = id
        self.subscribed_channels = set(random.sample(range(num_channels), np.random.randint(1, 11)))

In [3]:
# Каналы
class Channel:
    def __init__(self, id):
        self.id = id
        self.subscribers = 0
        self.connections = []
        self.is_sensitive = self.generate_post()

    def generate_post(self):
        return np.random.binomial(1, 0.3) == 1

    def choose_strategy(self, total_subs, expected_sub_growth):

        U_sensitive = self.calculate_payoff(total_subs, expected_sub_growth, sensitive=True)
        U_informative = self.calculate_payoff(total_subs, expected_sub_growth, sensitive=False)

        if self.is_sensitive:
            self.is_sensitive = U_sensitive > U_informative
        else:
            self.is_sensitive = U_informative >= U_sensitive

    def calculate_payoff(self, total_subs, expected_sub_growth, sensitive):
        
        if sensitive:
            payoff = (self.subscribers / total_subs) * (-self.subscribers) + \
                     (1 - (self.subscribers / total_subs)) * (self.subscribers + expected_sub_growth)
        else:
            payoff = (self.subscribers / total_subs) * self.subscribers
        
        return payoff

    def update_subscribers(self, nearby_channels, total_subs):
        if self.is_sensitive:
            for n_ch in nearby_channels:
                if not n_ch.is_sensitive:
                    gain_prob = self.subscribers / (n_ch.subscribers + self.subscribers)
                    subscribers_to_gain = int(gain_prob * n_ch.subscribers)
                    n_ch.subscribers -= subscribers_to_gain
                    self.subscribers += subscribers_to_gain

In [10]:
# Модель
class Model:
    def __init__(self, num_channels):
        self.channels = [Channel(i) for i in range(num_channels)]

    # Генерация сети
    def generate_connections(self):
        print("Сеть каналов:\n")
        for i, channel in enumerate(self.channels):
            for j in range(i + 1, len(self.channels)):
                if np.random.rand() < 0.5:
                    con = np.random.poisson(10)
                    channel.connections.append(self.channels[j])
                    print("Каналы", i, "&", j, "-", con, "\n")

                if np.random.rand() < 0.5:
                    con = np.random.poisson(10)
                    self.channels[j].connections.append(channel)
                    print("Каналы", j, "&", i, "-", con, "\n")

    # Процесс симуляции (10 раундов)
    def simulate(self, num_subscribers, regulator):
        
        subscribers = [Subscriber(i, len(self.channels)) for i in range(num_subscribers)]
        
        print("Стартовое распределение\n")
        for s in subscribers:
            for ch in s.subscribed_channels:
                self.channels[ch].subscribers += 1
        for channel in self.channels:
            print("Канал", channel.id, ":", channel.subscribers, "подп.\n")

        total_subs = sum(channel.subscribers for channel in self.channels)

        for round in range(10):
            print("Раунд", round + 1, "\n")
            for channel in self.channels:
                expected_sub_growth = 0
                channel.choose_strategy(total_subs, expected_sub_growth)
                if channel.is_sensitive:
                    print("Канал", channel.id, "публикует сенситивный пост.\n")
                else:
                    print("Канал", channel.id, "публикует информативный пост.\n")
                            
            regulator.monitor_channels(self.channels, total_subs)
                            
            for channel in self.channels:
                if channel.is_sensitive:
                    channel.update_subscribers(channel.connections, total_subs)
                            
            total_subs = sum(channel.subscribers for channel in self.channels)
            print("\n")
            for channel in self.channels:
                print("Канал", channel.id, ":", channel.subscribers, "подп.\n")

In [11]:
# Действия регулятора
class Regulator:
    def monitor_channels(self, channels, total_subs):
        for channel in channels:
            if channel.is_sensitive and np.random.rand() < (channel.subscribers / total_subs):
                print("Канал", channel.id, "заблокирован\n")
                channels.remove(channel)

In [12]:
# Пробую посмотреть промежуточные результаты на тесте симуляции
num_channels = 100
num_subscribers = 5000

# Генерим сначала сеть
model = Model(num_channels)
model.generate_connections()

# Смотрим саму модель
regulator = Regulator()
model.simulate(num_subscribers, regulator)

Сеть каналов:

Каналы 0 & 1 - 8 

Каналы 1 & 0 - 13 

Каналы 0 & 3 - 16 

Каналы 4 & 0 - 8 

Каналы 0 & 5 - 8 

Каналы 5 & 0 - 6 

Каналы 0 & 6 - 8 

Каналы 6 & 0 - 14 

Каналы 7 & 0 - 9 

Каналы 0 & 8 - 8 

Каналы 0 & 9 - 14 

Каналы 9 & 0 - 9 

Каналы 10 & 0 - 6 

Каналы 0 & 11 - 8 

Каналы 11 & 0 - 7 

Каналы 0 & 12 - 10 

Каналы 12 & 0 - 12 

Каналы 0 & 13 - 6 

Каналы 13 & 0 - 10 

Каналы 14 & 0 - 8 

Каналы 0 & 15 - 9 

Каналы 15 & 0 - 11 

Каналы 0 & 16 - 9 

Каналы 16 & 0 - 9 

Каналы 17 & 0 - 6 

Каналы 0 & 19 - 11 

Каналы 0 & 21 - 9 

Каналы 22 & 0 - 12 

Каналы 23 & 0 - 6 

Каналы 25 & 0 - 9 

Каналы 0 & 27 - 15 

Каналы 27 & 0 - 9 

Каналы 0 & 28 - 9 

Каналы 28 & 0 - 8 

Каналы 30 & 0 - 8 

Каналы 34 & 0 - 6 

Каналы 0 & 35 - 10 

Каналы 35 & 0 - 6 

Каналы 37 & 0 - 15 

Каналы 0 & 39 - 5 

Каналы 0 & 40 - 7 

Каналы 40 & 0 - 10 

Каналы 0 & 42 - 17 

Каналы 42 & 0 - 13 

Каналы 0 & 43 - 10 

Каналы 0 & 44 - 15 

Каналы 44 & 0 - 10 

Каналы 45 & 0 - 8 

Каналы 46 & 0 - 10